In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests

df = pd.read_csv('data.csv', index_col = 'ID')     #'P' = Patient    'H' = Healthy
df['class'] = df['class'].astype('category')

#Group by class and calculate the mean. Afterwards, transpose so the two classes are the only two columns
classes = df.groupby('class', observed= True).mean()
classes_t = classes.T

# Function to extract ending digits from index
def extract_task(df): 
    index = df.index.astype(str)
    tasks = index.str.extract(r'(\d{1,2})$', expand=False).astype(int)
    return tasks
classes_t['task'] = extract_task(classes_t)

<h4> MANN-WHITNEY U TEST </h4> 
With this test, we can see if there is a significant difference between the distribution of 'H' and 'P'. <br>
If the test is significant, we can reject H0 that there is no difference between the distributions of values between 'H' and 'P'

<h4> BONFERRONI TEST </h4>
The Bonferroni test corrects for the many pvalues that are generated between the observations. Since it is important that Type II errors get avoided, I chose for Bonferroni since this is a rather conservative test. 

In [2]:
df_p = df[df['class'] == 'P'] #Dataframe with people diagnosed with alzheimers
df_h = df[df['class'] == 'H'] #Dataframe with healthy people

p_values = list()
effect_sizes = list()

def cohens_d(x, y): #Calculating the effect size
    return (np.abs(np.mean(x) - np.mean(y))) / np.sqrt((np.std(x, ddof=1) ** 2 + np.std(y, ddof=1) ** 2) / 2)


for feature in df.columns[:-1]:
    data_p = df_p[feature]
    data_h = df_h[feature]
    
    value, p_val = stats.mannwhitneyu(data_p, data_h)
    p_values.append(p_val)
    
    d = cohens_d(df_p[feature], df_h[feature])
    effect_sizes.append(d)  

_, corrected_pvals, _, _ = multipletests(p_values, alpha = 0.05, method = 'bonferroni')

classes_t['difference_pval'] = corrected_pvals
classes_t['effect_size'] = effect_sizes
classes_t['significant_difference'] = (classes_t['difference_pval'] <= 0.05).astype(int)

classes_t = classes_t.rename(columns= {'H': 'H_mean', 'P': 'P_mean'})

display(classes_t.tail(5))

class,H_mean,P_mean,task,difference_pval,effect_size,significant_difference
num_of_pendown25,82.023529,89.483146,25,1.000000,0.273650,0
paper_time25,36448.823529,49471.235955,25,0.000059,0.728839,1
pressure_mean25,1721.033135,1542.248773,25,1.000000,0.576080,0
pressure_var25,160722.644250,165295.761342,25,1.000000,0.080493,0
total_time25,107617.294118,218246.168539,25,0.016017,0.225793,1


In [3]:
significant_features = classes_t[classes_t['significant_difference'] == 1]
significant_features = significant_features.sort_values('difference_pval')
display(significant_features.head())

significant_features_idx = significant_features.index
sig = df[significant_features_idx] #Dataframe with only the significant features 

display(sig.head())

class,H_mean,P_mean,task,difference_pval,effect_size,significant_difference
total_time23,11816.870588,17025.561798,23,6.062692e-14,0.220935,1
total_time15,14841.682353,53423.022472,15,2.127553e-12,0.878697,1
air_time23,7383.752941,9785.393258,23,2.147543e-12,0.105286,1
air_time15,9390.211765,43798.303371,15,2.944023e-11,0.833366,1
total_time17,34972.223529,74392.303371,17,6.999588e-11,0.496649,1


,total_time23,total_time15,air_time23,air_time15,total_time17,paper_time23,air_time17,paper_time17,total_time6,air_time16,...,mean_jerk_on_paper3,mean_acc_on_paper2,pressure_var21,mean_jerk_in_air25,mean_gmrt1,num_of_pendown7,mean_gmrt9,gmrt_on_paper22,gmrt_on_paper15,gmrt_on_paper23
ID,,,,,,,,,,,,,,,,,,,,,
id_1,16160,32384,10965,17354,43285,5195,26660,16625,7675,3730,...,0.017351,0.076406,91381.55699,0.141434,103.828754,4,690.253070,175.179965,57.804271,190.158460
id_2,29900,41200,14660,26535,103935,15240,54370,49565,30080,10650,...,0.015674,0.103937,149469.69830,0.049663,99.383459,6,56.617400,56.986142,49.014189,52.977313
id_3,13865,33695,7330,22345,50990,6535,27640,23350,5345,3265,...,0.011406,0.075103,54854.49989,0.178194,201.347928,5,198.257793,151.306239,85.956735,189.682156
id_4,13585,28465,7205,21890,49645,6380,27000,22645,29970,9850,...,0.011895,0.129888,105063.56350,0.113905,276.298223,7,158.085598,116.624172,120.452831,166.255725
id_5,10145,24360,5340,18575,37675,4805,22375,15300,11870,805,...,0.015170,0.081101,143524.84850,0.121782,184.636510,4,151.967684,144.301940,90.797232,157.823013


<h3> PCA </h3>
I applied PCA to reduce the number of column sin the dataset. I wanted to capture 99% of the variance, which led to almost halve of the significant columns. 

In [4]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range= (-1, 1))
pca = PCA(n_components= 0.99)
sig_scaled = scaler.fit_transform(sig)
sig_scaled_transformed = pca.fit_transform(sig_scaled)
pca_frame = pd.DataFrame(sig_scaled_transformed, index = sig.index)
pca_frame.head()

,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,82
ID,,,,,,,,,,,,,,,,,,,,,
id_1,0.145780,1.009747,-0.670994,-0.668427,-0.898119,-0.377622,0.993143,1.318765,1.025275,-0.046577,...,-0.068585,-0.182752,0.165853,0.077931,-0.095847,0.049297,0.115303,-0.035538,0.009996,0.105231
id_2,4.458862,0.309427,-0.252775,-0.513903,-0.026872,-1.160453,-1.132002,0.917680,-0.046371,-0.397825,...,-0.109880,-0.155270,-0.095353,0.086493,0.021690,0.074153,0.057186,0.091228,0.005370,0.119461
id_3,0.290307,-0.173949,-0.933445,0.345094,-0.042128,0.072288,-0.341775,-0.316597,0.097117,-0.061462,...,-0.174323,0.050960,-0.135475,-0.079350,0.039035,0.064734,-0.257369,0.314196,-0.109449,0.097987
id_4,2.023812,-0.596423,0.434598,0.037898,-0.197099,-0.496754,-0.376483,-0.130767,0.350506,-0.146561,...,-0.347328,0.073223,0.023111,-0.191515,-0.083507,0.131871,-0.165518,-0.006832,0.181772,-0.273191
id_5,0.846310,0.354092,-1.457208,-0.234651,-0.864111,-0.801228,-0.541422,0.358469,0.791681,0.585810,...,0.037641,-0.230070,-0.130213,0.253158,0.107898,0.122623,0.067777,0.042417,0.111316,0.058655


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

X_pca = pca_frame.values
y = le.fit_transform(df['class'])

X_pca_train, X_pca_test, y_train, y_test = train_test_split(X_pca, y, test_size= 0.3,random_state= 1, stratify= y, shuffle = True)

In [6]:
#BASELINE LOGISTIC REGRESSION MODEL 
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

pca_logreg = LogisticRegression(random_state= 1)
pca_logreg_scores = cross_val_score(pca_logreg, X_pca_train, y_train, cv = 5, scoring = 'roc_auc')
print(f'{pca_logreg.__class__.__name__} CV roc_auc score: {pca_logreg_scores.mean()}')

pca_logreg.fit(X_pca_train, y_train)
pca_logreg_pred = pca_logreg.predict(X_pca_test)
print(f'--- {pca_logreg.__class__.__name__} classification report ---')
print(classification_report(pca_logreg_pred, y_test))

LogisticRegression CV roc_auc score: 0.973970473970474
--- LogisticRegression classification report ---
              precision    recall  f1-score   support

           0       0.88      0.77      0.82        30
           1       0.74      0.87      0.80        23

    accuracy                           0.81        53
   macro avg       0.81      0.82      0.81        53
weighted avg       0.82      0.81      0.81        53



In [7]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

classifiers = [RandomForestClassifier(random_state= 1), GradientBoostingClassifier(random_state= 1), 
               SVC(random_state= 1), XGBClassifier(random_state = 1)]

for pca_clf in classifiers: 
    pca_clf_scores = cross_val_score(pca_clf, X_pca_train, y_train, cv = 5, scoring = 'roc_auc')
    print(f'{pca_clf.__class__.__name__} CV roc_auc score: {pca_clf_scores.mean()}')
    
    pca_clf.fit(X_pca_train, y_train)
    pca_clf_pred = pca_clf.predict(X_pca_test)
    print(f' --- {pca_clf.__class__.__name__} classification report ---')
    print(classification_report(pca_clf_pred, y_test))
    print()
    print()


RandomForestClassifier CV roc_auc score: 0.9600330225330225
 --- RandomForestClassifier classification report ---
              precision    recall  f1-score   support

           0       0.85      0.81      0.83        27
           1       0.81      0.85      0.83        26

    accuracy                           0.83        53
   macro avg       0.83      0.83      0.83        53
weighted avg       0.83      0.83      0.83        53



GradientBoostingClassifier CV roc_auc score: 0.923076923076923
 --- GradientBoostingClassifier classification report ---
              precision    recall  f1-score   support

           0       0.88      0.74      0.81        31
           1       0.70      0.86      0.78        22

    accuracy                           0.79        53
   macro avg       0.79      0.80      0.79        53
weighted avg       0.81      0.79      0.79        53



SVC CV roc_auc score: 0.966977466977467
 --- SVC classification report ---
              precision    recal

<h3> UNSUCCESFULLL </h3>
With the PCA approach, RF was barely able to beat the benchmark. Lets see how it performs without applying PCA.

<h2> WITHOUT PCA </h2>

In [8]:
X = scaler.fit_transform(sig)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, stratify= y, random_state= 1, shuffle = True)

In [9]:
logreg = LogisticRegression(random_state= 1)
logreg_scores = cross_val_score(logreg, X_train, y_train, cv = 5, scoring= 'roc_auc')
print(f'{logreg.__class__.__name__} CV roc_auc score: {cross_val_score(logreg, X_train, y_train, cv = 5).mean()}')

logreg.fit(X_train, y_train)
logreg_pred = logreg.predict(X_test)
print(f'--- {logreg.__class__.__name__} classification report ---')
print(classification_report(logreg_pred, y_test))

LogisticRegression CV roc_auc score: 0.8846666666666666
--- LogisticRegression classification report ---
              precision    recall  f1-score   support

           0       0.88      0.74      0.81        31
           1       0.70      0.86      0.78        22

    accuracy                           0.79        53
   macro avg       0.79      0.80      0.79        53
weighted avg       0.81      0.79      0.79        53



In [10]:
for clf in classifiers: 
    clf_scores = cross_val_score(clf, X_train, y_train, cv = 5, scoring = 'roc_auc')
    print(f'{clf.__class__.__name__} CV roc_auc score: {clf_scores.mean()}')
    
    clf.fit(X_train, y_train)
    clf_pred = clf.predict(X_test)
    print(f' --- {clf.__class__.__name__} classification report ---')
    print(classification_report(clf_pred, y_test))
    print()
    print()


RandomForestClassifier CV roc_auc score: 0.9598921911421912
 --- RandomForestClassifier classification report ---
              precision    recall  f1-score   support

           0       0.88      0.82      0.85        28
           1       0.81      0.88      0.85        25

    accuracy                           0.85        53
   macro avg       0.85      0.85      0.85        53
weighted avg       0.85      0.85      0.85        53



GradientBoostingClassifier CV roc_auc score: 0.9066142191142191
 --- GradientBoostingClassifier classification report ---
              precision    recall  f1-score   support

           0       0.88      0.79      0.84        29
           1       0.78      0.88      0.82        24

    accuracy                           0.83        53
   macro avg       0.83      0.83      0.83        53
weighted avg       0.84      0.83      0.83        53



SVC CV roc_auc score: 0.9709401709401708
 --- SVC classification report ---
              precision    rec

<h3> SUCCESFULL </h3>
All models managed to beat the baseline. This brings up the question, was it a poor PCA, or does the baseline work well with PCA? 

</h3> HYPERPARAMETER TUNING RANDOM FOREST AND XGBCLASSIFIER </h3>

In [11]:
from sklearn.model_selection import RandomizedSearchCV, RepeatedStratifiedKFold
from scipy.stats import randint, uniform 

cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 3, random_state= 1)

rf_params = {
    'n_estimators': randint(50, 150), 
    'max_depth': [3, 5, 7, None], 
    'min_samples_split': randint(2, 8), 
    'min_samples_leaf': randint(1, 4)
}
rf = RandomForestClassifier(random_state= 1)
rf_cv = RandomizedSearchCV(rf, param_distributions= rf_params, 
                           n_iter = 30, cv = cv, random_state= 1, n_jobs = -1)

rf_cv.fit(X_train, y_train)
print('Best RF params:', rf_cv.best_params_)

Best RF params: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 66}


In [12]:
xgb_params  ={
    'n_estimators': randint(50, 150), 
    'learning_rate': uniform(0.01, 0.1), 
    'max_depth': randint(2, 5), 
    'subsample': uniform (0.6, 0.4), 
    'colsample_bytree' : uniform(0.6, 0.4), 
    'reg_alpha': uniform(0, 0.5), 
    'reg_lambda': uniform(0, 0.5)
}

xgb = XGBClassifier(random_state = 1)
xgb_cv = RandomizedSearchCV(xgb, param_distributions= xgb_params, 
                            n_iter = 30, cv = cv, random_state = 1, n_jobs = -1)

xgb_cv.fit(X_train, y_train)
print('Best XGB params:', xgb_cv.best_params_)

Best XGB params: {'colsample_bytree': 0.6158734008262905, 'learning_rate': 0.09138763651723374, 'max_depth': 3, 'n_estimators': 137, 'reg_alpha': 0.15868120466108038, 'reg_lambda': 0.49430807720622444, 'subsample': 0.8318980876983187}


In [13]:
final_clf = [rf_cv.best_estimator_, xgb_cv.best_estimator_]

for clf in final_clf: 
    clf.fit(X_train, y_train)
    clf_pred = clf.predict(X_test)
    print(f'--- {clf.__class__.__name__} classification report ---')
    print(classification_report(clf_pred, y_test))
    print()
    print()

--- RandomForestClassifier classification report ---
              precision    recall  f1-score   support

           0       0.88      0.85      0.87        27
           1       0.85      0.88      0.87        26

    accuracy                           0.87        53
   macro avg       0.87      0.87      0.87        53
weighted avg       0.87      0.87      0.87        53



--- XGBClassifier classification report ---
              precision    recall  f1-score   support

           0       0.88      0.85      0.87        27
           1       0.85      0.88      0.87        26

    accuracy                           0.87        53
   macro avg       0.87      0.87      0.87        53
weighted avg       0.87      0.87      0.87        53





Both respectable scores. However, when I was playing around with this dataset, I found that a higher score could be obtained when not applying the Mann-Whitney test. This could hafe different reasons. Maybe the algorithms do better when they can decide on their own which classes are important. I will make a seperate file where I will test this too. 